### Gas Price Data
This notebook uses the EIA (US Energy Informaton Association) data to populate a data frame containing monthly gas prices for a sample of states for the year 2019 (query_year is a variable and can easily be changed)

In [ ]:
### Dependencies and Keys
import requests
import json
import pandas as pd

import cufflinks as cf
import plotly.graph_objects as go
import plotly.express as px 

from datetime import date
import datetime

# Import API key
from api_keys import eia_key
from api_keys import g_key


# EIA API Query Browser
EIA Data Sets > Petroleum > Prices > Weekly Retail Gasoline and Diesel Prices

API CALL TO USE:http://api.eia.gov/category/?api_key=YOUR_API_KEY_HERE&category_id=240690

Children Categories:
by Area
by Product

##### https://www.eia.gov/developer/

#### https://www.eia.gov/opendata/


In [ ]:
url= f"http://api.eia.gov/category/?api_key={eia_key}&category_id=711295"
print(url)

##### U.S. Regular All Formulations Retail Gasoline Prices, Monthly

In [ ]:
url = f"http://api.eia.gov/series/?api_key={eia_key}&series_id=PET.EMM_EPMR_PTE_NUS_DPG.M"
#print(url)
response = requests.get(url).json()
#print(response)
#print(response.keys())   # response and series

chart_title = response['series'][0]['name']
chart_data  = response['series'][0]['data']
#print(chart_data)
prices_df = pd.DataFrame(chart_data, columns=['Date', 'Average Price'])
#prices_df

### Date comes in as a 6 character string YYYYMM so we have to strip it out if we want to plot anything meaningful
* We can use datetime library

In [ ]:
# use pd.to_datetime to reformat the date
prices_df['Date'] = pd.to_datetime(pd.Series(prices_df['Date']), format="%Y%m")
prices_df.head()

In [ ]:
start_date = datetime.datetime(2019, 1, 1)
end_date = datetime.datetime(2019, 12, 31)

prices_df = prices_df[prices_df['Date'] > start_date]
prices_df = prices_df[prices_df['Date'] < end_date]
prices_df

#### Creating Charts using PLOTLY

In [ ]:
fig = px.line(prices_df, x="Date", y="Average Price", title=chart_title
              +f' between {start_date.date()} and {end_date.date()}')
fig.show()

### EIA API Query Browser 
### EIA Data Sets > Petroleum > Prices > Weekly Retail Gasoline and Diesel Prices

The EIA Tracks prices for the following Regions, States and Cities in the Unites states
* U.S.
* East Coast (PADD 1)
* New England (PADD 1A)
* Central Atlantic (PADD 1B)
* Lower Atlantic (PADD 1C)
* Midwest (PADD 2)
* Gulf Coast (PADD 3)
* Rocky Mountain (PADD 4)
* West Coast (PADD 5)
* West Coast less California
* _California_
* _Colorado_
* _Florida_
* _Massachusetts_
* _Minnesota_
* _New York_
* _Ohio_
* _Texas_
* _Washington_
* Boston
* Chicago
* Cleveland
* Denver
* Houston
* Los Angeles
* Miami
* New York City
* San Francisco
* Seattle

This Notebook applies queries by state so we will only use the state data for the 9 states in italic

###### Set up the States to be Queried

In [ ]:
eia_states = ['CA', 'CO', 'FL', 'MA', 'MN', 'NY', 'OH', 'TX', 'WA']

#### Average gas prices over all of USA

#### Loop through API Queries and chart by State

In [ ]:
# # states
by_city_df = pd.DataFrame(columns=['Date', 'Average Price'])

for state in eia_states:
    #print(state)
    series_id = f'PET.EMM_EPM0_PTE_S{state}_DPG.M' 
#     print(series_id)

    url = f"http://api.eia.gov/series/?api_key={eia_key}&series_id={series_id}"
    #print(url)
    response = requests.get(url).json()
#     #print(response)
#     #print(response.keys())   # response and series

    chart_title = response['series'][0]['name']
    chart_data  = response['series'][0]['data']
    #convert to dataframe
    prices_df = pd.DataFrame(chart_data, columns=['Date', 'Average Price'])
#   Convert to Date - overwriting date string
    prices_df['Date'] = pd.to_datetime(pd.Series(prices_df['Date']), format="%Y%m")
    prices_df = prices_df[prices_df['Date']>=start_date]
    prices_df = prices_df[prices_df['Date']<end_date]

# and PLOT 

    by_city_df  = by_city_df.merge(prices_df, on='Date',how='outer')
    by_city_df  = by_city_df.rename(columns={"Average Price_y": state})
    by_city_df  = by_city_df.rename(columns={"Average Price_x": state})
    by_city_df  = by_city_df.rename(columns={"Average Price": state})

by_city_df=by_city_df.dropna(how='all', axis=1)

by_city_df

In [ ]:
df = by_city_df
labels = df.columns[1:,]
fig = px.line(df, x="Date", y="CA", 
              title=f"Average Montly Gas Price by State ({start_date.date()} to {end_date.date()}) ")
for col in df.columns[1:,]:
    #print(col)
    fig.add_scatter(x=df['Date'],y=df[col], mode="lines", name=col)
fig

#### END GAS Data Gathering

## Do Monthly Gas Prices Influence The Number of National Park Visitor 

In [ ]:
# read in my file from previous notebook 

In [ ]:
# Pull the visitor data for the states of interest
# Plot visitor data against gas see if correlation


In [ ]:
# Read in the csv containing state centroid coordinates
centroids = pd.read_csv("./Resources/state_centroids.csv")
centroids.head(3)

In [ ]:
# Map State name to State Code
# create a dictionary of states and abbreviations
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}
### Make a dict the other way around if needed
# abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))


In [ ]:
# Replace centroids full State with its abbreviation 
centroids = centroids.replace({"State": us_state_abbrev})
#centroids

In [ ]:
gas_states_df = centroids[centroids['State'].isin(eia_states)].reset_index()
gas_states_df = gas_states_df.drop('index', axis=1)
gas_states_df

In [ ]:
## Get the parks in the states of interest
parks_data = pd.read_csv("./Resources/state_park_visitor_by_month_and_population.csv")
parks_data.head(2)

In [ ]:
# Remove parks in states we are not using 
park_states_df = parks_data[parks_data['StateCode'].isin(eia_states)].reset_index()
park_states_df = park_states_df.drop('index', axis=1)
park_states_df

### PLAYING WITH PLOTLY 

In [ ]:
df = park_states_df
labels = df.columns[1:,]
fig = px.line(df, x="Month", y="Visitors", 
              title=f"Average Montly Visitors Price by State ({start_date.date()} to {end_date.date()}) ")
for col in df.columns[1:,]:
    #print(col)
    fig.add_scatter(x=df['Date'],y=df[col], mode="lines", name=col)


In [ ]:
fig

In [ ]:
romo = park_states_df[park_states_df["UnitCode"]=='ROMO']['Visitors']
romo
fig = px.scatter(x=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], y=romo)

for unitcode in park_states_df["UnitCode"]:
    #print(col)
    y_values =  park_states_df[park_states_df["UnitCode"]==unitcode]['Visitors']
    fig.add_scatter(x=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],y=y_values, mode="lines", name=col)
fig

fig.show()